[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Scientific Programming

## Dynamic Programming - Piece Wise Linear Regression

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 01/05/2025 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0092DeepLearningResNet.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning

# Deep Learning

# Miscellaneous
from enum import auto, Enum, unique
import math
import os
from platform import python_version
import random

# Typing
from typing import Callable, Dict, Generator, List, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython


## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
vallToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Color Palettes
lMatPltLibclr   = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'] #<! Matplotlib default color palette
lFlexokiClr     = ['#D14D41', '#DA702C', '#D0A215', '#879A39', '#3AA99F', '#4385BE', '#8B7EC8', '#CE5D97'] #<! Flexoki (Obsidian) Main
lFlexokiSatClr  = ['#AF3029', '#BC5215', '#BC5215', '#66800B', '#24837B', '#205EA6', '#5E409D', '#A02F6F'] #<! Flexoki (Obsidian) Saturated
lFlexokiGrayClr = ['#100F0F', '#1C1B1B', '#282726', '#343331', '#403E3C', '#55524E', '#878580', '#CECDC3'] #<! Flexoki (Obsidian) Grayscale
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())


In [ ]:
# Constants



In [ ]:
# Courses Packages



In [ ]:
# General Auxiliary Functions


## Piece Wise Regression

Assume the following data samples: $\left\{ {x}_{i}, {y}_{i} \right\}_{i = 1}^{N}$:

![](https://i.imgur.com/3VaO2je.png)
<!-- ![](https://i.postimg.cc/G3KV8wWL/TMP.png) -->

Without prior knowledge on the _knots_, the breakpoints between segments, using _Linear Regression_ will fail.

* <font color='brown'>(**#**)</font> There are models to solve the problem using a Convex Optimization.

In [ ]:
# Parameters

fileUrl = r'https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/refs/heads/master/DataSets/PieceWiseLinearData.csv'

# Data
numMatrices = 10


### Linear Regression

This section shows the result of _Naive_ use of linear regression for the data.

In [ ]:
# Load Data

dfData = pd.read_csv(fileUrl)


In [ ]:
# Plot Linear Model

with plt.style.context('dark_background'):
    hF, hA = plt.subplots(figsize = (10, 4))
    sns.set_theme(style = 'ticks', context = 'talk')
    sns.regplot(data = dfData, x = 'x', y = 'y', ax = hA, order = 1, ci = None, label = 'Samples', scatter_kws = dict(color = lFlexokiClr[0]), line_kws = dict(color = lFlexokiClr[1], label = 'Linear Fit'))
    hA.legend()
    hA.set_title('Linear Model');

### Dynamic Programming

In order to solve the problem as a Dynamic Programming problem one must define a cost function.  
A simple measure to the quality of the regression over a section could the be the ${R}^{2}$ Score ([Coefficient of Determination](https://en.wikipedia.org/wiki/Coefficient_of_determination)).

The model should be the solution given an optimal solution to a sub problem.

## Problem Statement

Given a set of points $\left\{ {x}_{i}, {y}_{i} \right\}_{i = 1}^{N}$ with up to $L$ segments find the optimal segmentation of teh data such that the sum of ${R}^{2}$ over all segments is minimized.

* <font color='brown'>(**#**)</font> It is assumed that ${x}_{i + 1} > {x}_{i}$.

### Dynamic Programming Solution

 * Define ${S}_{L} \left( i, j \right)$ to be the optimal score with up to $L$ segments over the data samples $i$ to $j$.
 * Initialization $S \left( i, i + 1 \right) = 1$.
 * The objective $S \left( 1, N \right)$.


The cost $C \left( i, j \right)$ will be defined as the ${R}^{2}$ score over $i \to j$ samples.

Conceptually, to calculate $S \left( i, j \right)$ one should find the best split:

$$
{S}_{L} \left( i, j \right) = \min_{k \in \left\{ 1, 2, 3}} \left( {S}_{L - 1} \left( i, k \right) + {S}_{L - 1} \left( k + 1, j \right) \right) + c \left( L \right)
$$

Where $c \left( L \right)$ is the cost of a split at the $L$ level.

* <font color='brown'>(**#**)</font> The term $c \left( L \right)$ is a regularization term to promote solutions with minimal splits.

In [ ]:
def CalcMinCostChainMatMul( lMatDim: List[Tuple[int, int]], iStart: int, jEnd: int, lChainCost: List[List[int]] ) -> int:
    """
    Calculate the minimum cost of a chain of matrix multiplication using recursion.
    The function takes a list of tuples, where each tuple represents the dimensions of a matrix.
    Input:
        - lMatDim: A list of tuples contains the dimensions of matrices (Rows, Columns).
    Output: 
        minCost: Minimum cost of matrix multiplication.
        m: A 2D list that stores the split points for optimal parenthesization.
    Example:
        ```python
        # Define the dimensions of matrices
        lMatDim = [(10, 20), (20, 30), (30, 40)]
        lChainCost  = [[-1] * len(lMatDim) for _ in range(len(lMatDim))] #<! Initialize the chain cost matrix
        # Calculate the minimum cost and split points
        minCost = CalcMinCostChainMatMul(lMatDim, 0, len(lMatDim) - 1, lChainCost)
        # Print the results
        print(f'Minimum Cost: {minCost}')
        print(f'Chain Cost: {lChainCost}')
        ```
    """

    # Use recursion to calculate the minimum cost of matrix multiplication
    if iStart == jEnd:
        return 0
    if lChainCost[iStart][jEnd] != -1:
        return lChainCost[iStart][jEnd]
    minCost = float('inf')
    for k in range(iStart, jEnd):
        splitCost = CalcMinCostChainMatMul(lMatDim, iStart, k, lChainCost) + CalcMinCostChainMatMul(lMatDim, k + 1, jEnd, lChainCost) + CalcMatMulOps((lMatDim[iStart][0], lMatDim[k][1]), (lMatDim[k + 1][0], lMatDim[jEnd][1]))
        if splitCost < minCost:
            minCost = splitCost
            lChainCost[iStart][jEnd] = minCost
    return minCost

In [ ]:
# Check the Function 001
lMatDim = [(10, 20), (20, 30), (30, 40)]
lChainCost  = [[-1] * len(lMatDim) for _ in range(len(lMatDim))]
minCost = CalcMinCostChainMatMul(lMatDim, 0, len(lMatDim) - 1, lChainCost)
minCost #<! Should be 18550

In [ ]:
# Check the Function 002
lMatDim = [(10, 20), (20, 30), (30, 40), (40, 5)]
lChainCost  = [[-1] * len(lMatDim) for _ in range(len(lMatDim))]
minCost = CalcMinCostChainMatMul(lMatDim, 0, len(lMatDim) - 1, lChainCost)
minCost #<! Should be 11955

In [ ]:
# Check the Function 003
lMatDim = [(3, 10), (10, 20), (20, 30), (30, 40), (40, 5), (5, 90), (90, 2), (2, 150)]
lChainCost  = [[-1] * len(lMatDim) for _ in range(len(lMatDim))]
minCost = CalcMinCostChainMatMul(lMatDim, 0, len(lMatDim) - 1, lChainCost)
minCost 

* <font color='red'>(**?**)</font> What's the purpose of the `lChainCost` variable?  
  Comment the branch `if lChainCost[iStart][jEnd] != -1:` and measure run time.
* <font color='red'>(**?**)</font> Does the implementation fully solve the problem?

<!-- 
1. It is a simple memoization mechanism.
2. One can not infer the order of calculation easily.
 -->

### Dynamic Programming

The _Dynamic Programming_ solution should improve the recursion in 2 ways:

 * Use memoization to avoid repeating calculations.
 * Being able to restore the path of the optimal solution.

In order to do so, the calculation is done on a graph defined by a matrix: $M \left( i, j \right) = \min_{k} \left( M \left( i, k \right) + M \left( k + 1, j \right) + \operatorname{Cost} \left( {d}_{i}, {d}_{k}, {d}_{j + 1} \right) \right)$.

Conceptually, the matrix of cost $\boldsymbol{M}$ is as following:

![](https://i.imgur.com/06g4yBX.png)
<!-- ![](https://i.postimg.cc/26bnfL0g/Diagrams-Matrix-Mul-003.png) -->

Each chain depends on the calculation of _sub chains_:

![](https://i.imgur.com/UvKopaG.png)
<!-- ![](https://i.postimg.cc/52r0W7DN/Diagrams-Matrix-Mul-004.png) -->

Which forces a certain path for the calculation (Order of calculation by the number of the arrow):

![](https://i.imgur.com/r1DMP2z.png)
<!-- ![](https://i.postimg.cc/Y0X7fjHq/Diagrams-Matrix-Mul-005.png) -->

* <font color='brown'>(**#**)</font> The calculation calculates all chains of length 1 (All pairs). Then the optimal chains of 3, then of 4, etc...



In [ ]:
def CalcMinCostChainMatMul( lMatDim: List[Tuple[int, int]] ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculate the minimum cost of matrix multiplication using dynamic programming.
    The function takes a list of tuples, where each tuple represents the dimensions of a matrix.
    Input:
        - lMatDim: A list of tuples contains the dimensions of matrices (Rows, Columns).
    Output: 
        minCost: Minimum cost of matrix multiplication.
        m: A 2D list that stores the split points for optimal parenthesization.
    Example:
        ```python
        # Define the dimensions of matrices
        matDim = [(10, 20), (20, 30), (30, 40)]
        # Calculate the minimum cost and split points
        minCost, splitPoints = CalcMinCostChainMatMul(matDim)
        print(f'Minimum Cost: {minCost}')
        print(f'Split Points: {splitPoints}')
        ```
    """
    numMat = len(lMatDim)
    mM     = np.full((numMat, numMat), -1, dtype = int) #<! Cost matrix
    np.fill_diagonal(mM, 0) #<! Fill the diagonal with 0
    mS     = np.zeros((numMat, numMat), dtype = int) #<! Split matrix

    # Starts with chains of length 1 and then increase the length of the chain
    for ll in range(1, numMat): #<! ll = length of the chain
        for iStart in range(numMat - ll):
            jEnd = iStart + ll
            mM[iStart][jEnd] = 9223372036854775807 #<! Set to maximum value (64 Bit Signed Integer)
            for k in range(iStart, jEnd):
                splitCost = mM[iStart][k] + mM[k + 1][jEnd] + CalcMatMulOps((lMatDim[iStart][0], lMatDim[k][1]), (lMatDim[k + 1][0], lMatDim[jEnd][1]))
                if splitCost < mM[iStart][jEnd]:
                    mM[iStart][jEnd] = splitCost
                    mS[iStart][jEnd] = k
    return mM, mS #<! Return the minimum cost and the split points


In [ ]:
# Check the Function 001
lMatDim = [(10, 20), (20, 30), (30, 40)]
mM, mS = CalcMinCostChainMatMul(lMatDim)
print(mM) #<! Look at the top right value
print(mS)

In [ ]:
# Check the Function 002
lMatDim = [(10, 20), (20, 30), (30, 5), (5, 40), (40, 90)]
mM, mS = CalcMinCostChainMatMul(lMatDim)
print(mM)
print(mS)

In [ ]:
# The Path (A * B * C * D * E -> (A * (B * C)) * (D * E))
CalcMatMulOps((20, 30), (30, 5)) + CalcMatMulOps((10, 20), (20, 5)) + CalcMatMulOps((5, 40), (40, 90)) + CalcMatMulOps((10, 5), (5, 90)) 

* <font color='green'>(**@**)</font> Write a function to parse the `mS` matrix into order of calculation.